In [1]:
import numpy as np
import json
import re
from settings import settings
from collections import defaultdict, Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import PyPDF2
from scipy.sparse.linalg import svds
from sklearn.preprocessing import normalize


def remove_punctuation(text):
    punctuation_pattern = r"[^\w\s]"
    return re.sub(punctuation_pattern, "", text)


def extract_tokens_from_regular_input(query):
    return remove_punctuation(query.lower()).strip().split()


def extract_tokens_from_file_input(file):
    pdf_read = PyPDF2.PdfReader(file)
    resume_text = ""
    for page in pdf_read.pages:
        resume_text += page.extract_text() or ""
    return extract_tokens_from_regular_input(remove_punctuation(resume_text))


def extract_tokens_from_docs(doc):
    attrs = [doc["company"], doc["role"], doc["country"], doc["city"], doc["skills"]]
    return remove_punctuation(" ".join(attrs).lower())


def tokenize_docs(doc):
    return doc.split()


def construct_invertex_index(vectorizer, tfidf_matrix, n_components):
    feature_names = vectorizer.get_feature_names_out()
    print(feature_names)
    terms_indices = {
        term: idx for idx, term in enumerate(feature_names) if idx < n_components
    }
    inverted_index = defaultdict(list)
    rows, cols = tfidf_matrix.nonzero()
    for row, col in zip(rows, cols):
        # if col < n_components:  # Only include columns within the reduced dimensions
        inverted_index[feature_names[col]].append((row, tfidf_matrix[row, col]))
    return terms_indices, inverted_index


def construct_term_idf_map(vectorizer):
    return dict(zip(vectorizer.get_feature_names_out(), vectorizer.idf_))


def construct_query_tfidf(query, idf_map):
    tokens = Counter(extract_tokens_from_regular_input(query))
    return {
        term: freq * idf_map.get(term, 0)
        for term, freq in tokens.items()
        if term in idf_map
    }


def compute_cosine_scores(query):
    query_tfidf = vectorizer.transform([query]).toarray()
    query_vec = normalize(np.dot(query_tfidf, words_compressed)).squeeze()
    sims = docs_compressed_normed.dot(query_vec)
    asort = np.argsort(-sims)[:200]
    return [(i, sims[i]) for i in asort[1:]]



def compute_greatest_sim(query):
    query_tfidf = vectorizer.transform([query]).toarray()
    query_vec = normalize(np.dot(query_tfidf, words_compressed)).squeeze()
    sims = docs_compressed_normed.multiply(query_vec)

def closest_projects_to_word(word_in, k=5):
    if word_in not in word_to_index:
        return []
    sims = docs_compressed_normed.dot(
        words_compressed_normed[word_to_index[word_in], :]
    )
    asort = np.argsort(-sims)[: k + 1]
    return [(i, documents[i][0], sims[i]) for i in asort[1:]]


# Load data and prepare TF-IDF and SVD
with open(settings.data_file_path, "r") as file:
    data = json.load(file)
documents = data.get("job_postings")

vectorizer = TfidfVectorizer(tokenizer=tokenize_docs, stop_words="english")

td_matrix = vectorizer.fit_transform(
    [extract_tokens_from_docs(doc) for doc in documents]
)

docs_compressed, s, words_compressed = svds(td_matrix, k=300)
words_compressed = words_compressed.transpose()

word_to_index = vectorizer.vocabulary_
index_to_word = {i: t for t, i in word_to_index.items()}

words_compressed_normed = normalize(words_compressed, axis=1)


td_matrix_np = td_matrix.transpose().toarray()
td_matrix_np = normalize(td_matrix_np)


docs_compressed_normed = normalize(docs_compressed)


In [2]:
from index import documents, compute_cosine_scores, extract_tokens_from_file_input


def group_postings_by_company(postings):
    groups = {}

    for p in postings:
        company = p["company"]

        if company in groups:
            groups[company]["postings"].append(p)
        else:
            groups[company] = {
                "name": company,
                "description": p["description"],
                "rating": p["rating"],
                "happiness": p["happiness"],
                "postings": [p],
            }

    return groups


def select_similar_k_docs(doc_scores_map, k=25):
    res = []

    for doc, score in doc_scores_map:
        document = documents[doc]
        document["score"] = score
        res.append(document)

        if k == 0:
            break

    return res


def get_postings_regular_input(text, k=25):
    cosine_scores = compute_cosine_scores(text)
    return select_similar_k_docs(cosine_scores, k)


def get_postings_file_input(file, k=25):
    file_tokens = extract_tokens_from_file_input(file)
    cosine_scores = compute_cosine_scores(" ".join(file_tokens))
    return select_similar_k_docs(cosine_scores, k)


In [247]:
query = "Proficiency in one or more backend programming languages (e.g., Java, Python, Node.js, Ruby) API development Database management (SQL or NoSQL) Server management and deployment Security best practices Knowledge of web frameworks (e.g., Express, Django)"

In [248]:
# query = "Highly skilled and compassionate Registered Nurse with over 7 years of experience in various healthcare settings. Proficient in providing comprehensive patient care, including assessment, planning, and implementation of nursing interventions. Strong ability to communicate effectively with patients, families, and interdisciplinary teams to ensure optimal patient outcomes. Experienced in managing diverse patient populations, ranging from pediatrics to geriatrics, with a focus on delivering evidence-based care. Proven track record of excellence in clinical skills, including medication administration, wound care, and patient education. Dedicated to continuous professional development and staying abreast of the latest advancements in nursing practice. Seeking to leverage my expertise and passion for patient care in a dynamic healthcare environment."

In [249]:
#Start with a query
query

'Proficiency in one or more backend programming languages (e.g., Java, Python, Node.js, Ruby) API development Database management (SQL or NoSQL) Server management and deployment Security best practices Knowledge of web frameworks (e.g., Express, Django)'

In [250]:
#Function to allow us to compare each individual job with the query by creating large array of repeat queries
def repeat_vector(vector, length):
    # Reshape the vector to be a row vector
    vector = np.array(vector).reshape(1, -1)
    
    # Use broadcasting to repeat the vector
    repeated_array = np.tile(vector, (length, 1))
    
    return repeated_array

In [251]:
#Create TFidf
query_tfidf = vectorizer.transform([query]).toarray()
query_vec = normalize(np.dot(query_tfidf, words_compressed)).squeeze()

#Create repeated query vector to analyze how much each index contributes in the dot product (cosine score)
length = len(docs_compressed_normed)
repeated_query_vec = repeat_vector(query_vec, length)

#Multiply by each document
sims = docs_compressed_normed *repeated_query_vec

#Extract out the indices that contribute the most to the cosine score and their values
largest_indices = np.argsort(-sims, axis=1)[:, 0:100]
largest_values = np.sort(-sims, axis=1)[:, 0:100]


In [252]:
#Now, extract the relation between the SVD terms and our query
#Extract all the words in the tokenizer
words = vectorizer.get_feature_names_out()
#Find out which value in our query is most related to each SVD index
max_indices = np.argmax(np.expand_dims(query_tfidf.squeeze(), axis=-1) * words_compressed_normed, axis=0)
#This creates our dictionary to display our statistics
word_representations = []
for i in max_indices:
    word_representations.append(words[i])

In [253]:
#Now calculuate the words that link the query to each job posting, the most similar words between them
most_sim_words = []
sim_scores = []
k = 5 #top k keys

n,m = largest_indices.shape
for i in range(n):
    #Track contribution of each word in their similarity score
    contribution_per_index = {word: 0 for word in word_representations}
    for j in range(m):
         contribution_per_index[word_representations[largest_indices[i,j]]] -= largest_values[i,j]   
    
    #Find top k related words
    top_k_words = sorted(contribution_per_index, key=contribution_per_index.get, reverse=True)[:k]
    
    #Track all the scores, this will show the percentage the top k related words contribute to the cosine score
    sorted_values = sorted(contribution_per_index.values(), reverse=True)
    
    #Track
    most_sim_words.append(top_k_words)
    sim_scores.append(sorted_values)

In [254]:
#Calculuate similarity scores
sims = docs_compressed_normed.dot(query_vec)
p = 10
#Find the top p postings
asort = np.argsort(-sims)[:p]
#Get their most similar words
result = [most_sim_words[i] for i in asort]
#Get the contribution of each word
result_scores = [sim_scores[i] for i in asort]

In [255]:
result

[['backend', 'server', 'database', 'deployment', 'security'],
 ['backend', 'server', 'database', 'security', 'deployment'],
 ['backend', 'server', 'database', 'security', 'deployment'],
 ['backend', 'server', 'database', 'security', 'deployment'],
 ['backend', 'server', 'database', 'security', 'deployment'],
 ['backend', 'server', 'database', 'deployment', 'security'],
 ['backend', 'server', 'database', 'security', 'deployment'],
 ['backend', 'server', 'database', 'security', 'deployment'],
 ['backend', 'server', 'database', 'deployment', 'security'],
 ['backend', 'server', 'database', 'security', 'deployment']]

In [256]:
result_scores

[[0.1601060370059522,
  0.11908065757896903,
  0.10177369710211472,
  0.07142938948687641,
  0.07119935517186723,
  0.05913440597595163,
  0.04737266197566474,
  0.033022502045955054,
  0.025541165033169968,
  0.022127696984439976,
  0.019341526841560486,
  0.01916573554683185,
  0.018409253369231104,
  0.01426039080132016,
  0.011508946898624977,
  0.008986392198826524,
  0.005280428417966424],
 [0.15750656213589675,
  0.11616620530248593,
  0.0980847552235513,
  0.06998319699119955,
  0.06951988776282732,
  0.059311936744875365,
  0.04522978538051666,
  0.031009700396964986,
  0.026241019999903005,
  0.02090882894045512,
  0.02018316472395842,
  0.018881900080560406,
  0.018120147698438716,
  0.01540304714512067,
  0.012436810127287508,
  0.00873939661704882,
  0.004892574592321777],
 [0.1583130881609707,
  0.12094163727918449,
  0.09821505333332779,
  0.06521616969234668,
  0.06464304942730945,
  0.05573087451667834,
  0.041657393045830264,
  0.03080156092930858,
  0.023079520963240

In [257]:
# Find the sum of total contributions for each word
sums = np.sum(np.abs(result_scores), axis=1)

# Calculate the percentage contribution of each word
percentages = (result_scores / sums[:, np.newaxis]) * 100


In [258]:
#Display top k
percentages = percentages[:,:k]
percentages

array([[19.82147584, 14.74244458, 12.5998052 ,  8.84311388,  8.81463513],
       [19.87166319, 14.65599702, 12.37476835,  8.82936241,  8.77090945],
       [20.4166353 , 15.59707621, 12.66617276,  8.41051595,  8.33660426],
       [20.29268807, 14.77966323, 12.69403628,  9.05985142,  8.76416448],
       [20.135472  , 14.75170246, 12.26073609,  8.75425183,  8.71989701],
       [20.37448537, 14.7076827 , 12.62105884,  9.31434808,  8.6419129 ],
       [20.55775573, 14.6482727 , 12.90822209,  8.93381977,  8.73308698],
       [20.59678429, 14.74641405, 13.11105246,  9.06594727,  9.01036866],
       [20.88048172, 14.25432831, 11.46353702,  9.66605405,  8.26395269],
       [18.27155595, 14.79852674, 13.08093458,  8.31956862,  8.22960309]])

In [259]:
result

[['backend', 'server', 'database', 'deployment', 'security'],
 ['backend', 'server', 'database', 'security', 'deployment'],
 ['backend', 'server', 'database', 'security', 'deployment'],
 ['backend', 'server', 'database', 'security', 'deployment'],
 ['backend', 'server', 'database', 'security', 'deployment'],
 ['backend', 'server', 'database', 'deployment', 'security'],
 ['backend', 'server', 'database', 'security', 'deployment'],
 ['backend', 'server', 'database', 'security', 'deployment'],
 ['backend', 'server', 'database', 'deployment', 'security'],
 ['backend', 'server', 'database', 'security', 'deployment']]

In [260]:
#Now graph using a spider plot
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [261]:
data = {
    'word': result[0],
    'percent': percentages[0,0:5]
}
df = pd.DataFrame(data)

In [262]:
layout = go.Layout(title='Contribution Percentages of Top 5 Words in Your Resume')
# Example data
categories = ['A', 'B', 'C', 'D', 'E']
values = [4, 3, 2, 5, 4]

fig = go.Figure(layout=layout)
fig.add_trace(
    go.Scatterpolar(
        r=df['percent'],
        theta=df['word'],
        fill='toself',
        fillcolor='lightgreen',
        name='Contribution Percentages'
    )
)


fig.show()